# Guide: run async programs

In this guide we will look into an ability of QuantumServerless running asynchronous programs as async jobs.

Let's import all necessary dependencies and create `QuantumServerless` instance.

In [1]:
from quantum_serverless import QuantumServerless, run_qiskit_remote, get, Program

In [2]:
serverless = QuantumServerless({
    "providers": [{
        "name": "docker",
        "compute_resource": {
            "name": "docker",
            "host": "localhost",
        }
    }]
})
serverless

<QuantumServerless | providers [local, docker]>

> NOTE: important thing to remember is that async jobs are only available with real cluster and will not work in local machine. 

In this example we will have head node running on inside docker on localhost

---

Now we have all prep steps figured out. Next step will be defining script for the job itself.

Inside the script we will be using the same principles that we showed in other tutorials (creating QuantumServerless instance, annotating functions and getting results)

```python
# program.py
from quantum_serverless import QuantumServerless, run_qiskit_remote, get


@run_qiskit_remote()
def ultimate():
    return 42


with QuantumServerless():
    result = get([
        ultimate()
        for _ in range(100)
    ])
    
print(result)
```

[Program file](./program.py) will be located in the same directory as our notebook.


Let's run our script/job using `run_program` function provided by QuantumServerless.
Run program accepts `Program` instance.

In [7]:
program = Program(
    name="awesome_program",
    entrypoint="program.py",
    working_dir="./",
    description="description of a program",
    version="0.0.1"
)

job = serverless.run_program(program)
job

<Job | fqs_9ae47383-3c21-41ab-84f6-61a969a68b06>

You will get `Job` object as result, which you can query for `status` and `logs` to see status of your job and logs.

In [15]:
job.status()

<JobStatus.SUCCEEDED: 'SUCCEEDED'>

In [16]:
job.logs()

'/home/ray/anaconda3/lib/python3.7/site-packages/ray/_private/worker.py:983: UserWarning: len(ctx) is deprecated. Use len(ctx.address_info) instead.\n  warnings.warn("len(ctx) is deprecated. Use len(ctx.address_info) instead.")\n[42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42]\n'

In [17]:
# recover job by id
recovered_job = serverless.get_job_by_id(job.job_id)